# Projeto: Esgotamento Sanitário na Região Nordeste (2010)

### Autores:

- Gabriel Igor Queiroz Costa - gabriel.igorq@gmail.com
- Erick de Oliveira SIlva - erickoliveira.eos@gmail.com

Inicialmente, devemos executar as células abaixo para realizarmos o upload dos arquivos necessários para a execução das linhas de código. Para isso, siga as instruções:

- Clone o repositório para o seu sistema local;
- Execute a célula abaixo;
- Clique em **"Browse"** e selecione o arquivo **dadosNE.zip**;
- Aguarde o arquivo carregar e execute as demais células abaixo até o tópico de Introdução



In [95]:
# Uploading files from your local file system

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dadosNE.zip to dadosNE.zip
User uploaded file "dadosNE.zip" with length 3392485 bytes


In [96]:
!unzip dadosNE.zipn

Archive:  dadosNE.zip
replace data/esg_AL.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/esg_BA.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/esg_CE.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/esg_MA.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/esg_PB.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/esg_PE.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/esg_PI.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/esg_RN.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/esg_SE.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace geojson/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace geojson/geojs-21-mun.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace geojson/geojs-22-mun.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace geojson/geojs-23-mun.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace geojson/geojs-24-mun.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace geojson/geojs-25-m

In [0]:
!pip install folium shapely

In [0]:
import os
import folium
import json
import pandas as pd
from branca.colormap import linear
import numpy as np
from shapely.geometry import Polygon
from shapely.geometry import Point
from numpy import random

# 1 - Introdução 

 <p> O Instituto Brasileiro de Geografia e Estatística disponibiliza em seu site diversos dados relativos ao Brasil. Nesse notebook nós acessamos dados referentes  ao percentual de esgotamento sanitário de cada um dos estados do Nordeste no ano de 2010 para gerar mapas temáticos usando o choropleth e analisar se existem grandes diferenças entre os estado dessa mesma região. <p>
  
<p>Segundo a Organização Mundial de Saúde (OMS), saneamento é o controle de todos os fatores do meio físico do homem que exercem ou podem exercer efeitos nocivos sobre o bem estar físico, mental e social. É o conjunto de medidas adotadas em um local para melhorar a vida e a saúde dos habitantes, impedindo que fatores físicos de efeitos nocivos possam prejudicar as pessoas no seu bem-estar físico mental e social. No Brasil, esse conceito está previsto pela Lei Federal nº 11.445/07.<p>

## 1.1 Importando os datasets

In [0]:
dataRN = pd.read_csv("esg_RN.csv")
dataPB = pd.read_csv("esg_PB.csv")
dataPE = pd.read_csv("esg_PE.csv")
dataCE = pd.read_csv("esg_CE.csv")
dataMA = pd.read_csv("esg_MA.csv")
dataPI = pd.read_csv("esg_PI.csv")
dataAL = pd.read_csv("esg_AL.csv")
dataSE = pd.read_csv("esg_SE.csv")
dataBA = pd.read_csv("esg_BA.csv")

Após importar os dados, podemos concatená-los em um único dataset:

In [0]:
data = pd.concat([dataRN, dataPB, dataPE, dataCE, dataMA, dataPI, dataAL, dataSE, dataBA], ignore_index=True)

Para alterar o nome das colunas:

In [0]:
data.columns = ["Cidade", "Esgotamento"]

In [42]:
data.head()

,Cidade,Esgotamento
0,Acari,81.4
1,Açu,13.3
2,Afonso Bezerra,51.3
3,Água Nova,7.1
4,Alexandria,56.1


# 2 -  Importando os arquivos GeoJson

Existe um projeto  [Geojson](http://geojson.org/) contendo os perímetros de todos os municípios brasileiros agrupados por estado.(Fonte: https://github.com/tbrugz/geodata-br)

Para a região Nordeste, temos os seguintes dados:

* AL / Alagoas - [geojson/geojs-27-mun.json](geojson/geojs-27-mun.json)
* BA / Bahia - [geojson/geojs-29-mun.json](geojson/geojs-29-mun.json)
* CE / Ceará - [geojson/geojs-23-mun.json](geojson/geojs-23-mun.json)
* MA / Maranhão - [geojson/geojs-21-mun.json](geojson/geojs-21-mun.json)
* PB / Paraíba - [geojson/geojs-25-mun.json](geojson/geojs-25-mun.json)
* PE / Pernambuco - [geojson/geojs-26-mun.json](geojson/geojs-26-mun.json)
* PI / Piauí - [geojson/geojs-22-mun.json](geojson/geojs-22-mun.json)
* RN / Rio Grande do Norte - [geojson/geojs-24-mun.json](geojson/geojs-24-mun.json)
* SE / Sergipe - [geojson/geojs-28-mun.json](geojson/geojs-28-mun.json)

In [0]:
# Buscando arquivos em geojson/geojs-xx-mun.json
MA = os.path.join('geojson', 'geojs-21-mun.json')
PI = os.path.join('geojson', 'geojs-22-mun.json')
CE = os.path.join('geojson', 'geojs-23-mun.json')
RN = os.path.join('geojson', 'geojs-24-mun.json')
PB = os.path.join('geojson', 'geojs-25-mun.json')
PE = os.path.join('geojson', 'geojs-26-mun.json')
AL = os.path.join('geojson', 'geojs-27-mun.json')
SE = os.path.join('geojson', 'geojs-28-mun.json')
BA = os.path.join('geojson', 'geojs-29-mun.json')

nordeste = os.path.join('geojson', 'geojs-NE-mun.json')

# load the data and use 'latin-1'encoding because the accent
geo_json_NE = json.load(open(nordeste,encoding='latin-1'))

geo_json_MA = json.load(open(MA,encoding='latin-1'))
geo_json_PI = json.load(open(PI,encoding='latin-1'))
geo_json_CE = json.load(open(CE,encoding='latin-1'))
geo_json_RN = json.load(open(RN,encoding='latin-1'))
geo_json_PB = json.load(open(PB,encoding='latin-1'))
geo_json_PE = json.load(open(PE,encoding='latin-1'))
geo_json_AL = json.load(open(AL,encoding='latin-1'))
geo_json_SE = json.load(open(SE,encoding='latin-1'))
geo_json_BA = json.load(open(BA,encoding='latin-1'))

In [0]:
# http://cidades.ibge.gov.br/painel/historico.php?codmun=241030
# Presidente Juscelino city changes your name to Serra Caiada
geo_json_RN['features'][112]['properties']['description'] = 'Serra Caiada'
geo_json_RN['features'][112]['properties']['name'] = 'Serra Caiada'

In [100]:
# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=5,
    tiles='Stamen Terrain'
)

# Configure geojson layer

folium.GeoJson(geo_json_NE).add_to(m)

m

# 3 - Criando mapas temáticos com o Choropleth

In [63]:
data.head()

,Cidade,Esgotamento
0,Acari,81.4
1,Açu,13.3
2,Afonso Bezerra,51.3
3,Água Nova,7.1
4,Alexandria,56.1


In [110]:
# colormap yellow and green (YlGn)
colormap = linear.YlGn_03.scale(
    data["Esgotamento"].min(),
    data["Esgotamento"].max())

print(colormap(5000.0))

colormap

#31a354


In [0]:
# Converter para dicionário
esg_dict = data.set_index('Cidade')['Esgotamento']
esg_dict[:100]

In [106]:
# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=7,
    tiles='Stamen Terrain'
)

# create a threshold of legend
threshold_scale = np.linspace(data['Esgotamento'].min(),
                              data['Esgotamento'].max(), 6, dtype=int).tolist()

m.choropleth(
    geo_data=geo_json_NE,
    data=data,
    columns=['Cidade', 'Esgotamento'],
    key_on='feature.properties.description',
    fill_color='BuGn',
    legend_name='Esgotamento sanitário adequado (2010)',
    highlight=True,
    threshold_scale = threshold_scale
)

m



## 3.1 Saneamento no RN

In [0]:
dataRN.columns = ['Cidade', 'Esgotamento']

In [121]:
test = dataRN.sort_values(['Esgotamento'],ascending=False)
test

,Cidade,Esgotamento
157,Timbaúba dos Batistas,97.3
33,Currais Novos,82.2
0,Acari,81.4
122,Santa Cruz,81.2
22,Caicó,76.3
153,Tangará,75.4
165,Viçosa,73.4
146,Serra Negra do Norte,73.3
62,Jardim do Seridó,73.1
75,Lucrécia,72.4


In [111]:
# Create a map object
rn = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=7,
    tiles='Stamen Terrain'
)

# create a threshold of legend
threshold_scale = np.linspace(data['Esgotamento'].min(),
                              data['Esgotamento'].max(), 6, dtype=int).tolist()

rn.choropleth(
    geo_data=geo_json_RN,
    data=data,
    columns=['Cidade', 'Esgotamento'],
    key_on='feature.properties.description',
    fill_color='YlGn',
    legend_name='Esgotamento sanitário adequado no RN (2010)',
    highlight=True,
    threshold_scale = threshold_scale
)

rn

# 4 - Conclusão